# KGCOVID19 Animation
Sometimes, expecially when preparing a presentation for a conference or your work colleagues, a good animation can say more than a thousand words.

For this reason, we have prepared a straighforward way to create animations for a number of tasks using GRAPE that, through subsampling, can be executed on graphs of arbitrary size.

In this brief tutorial, we will show how to get a KGCOVID19 embedding using First-order LINE, and then we will use TSNE decoposition to reduce its dimensionality and plot it into a short video.

The resulting WEBM can be converted using one of many services and can be directly incorporated in Google Slides.

## Retrieving KGCOVID19
First, we retrieve KGCOVID19:

In [1]:
from grape.datasets.kghub import KGCOVID19

graph = KGCOVID19()

Then, let's take a look at its graph report:

In [2]:
graph

<div class="graph-report"><style>.graph-report li {margin: 0.5em 0 0.5em 0;}.graph-report .paragraph {text-align: justify;word-break: break-all;}.graph-report .small-columns {column-count: 4;column-gap: 2em;}.graph-report .medium-columns {column-count: 3;column-gap: 2em;}.graph-report .large-columns {column-count: 2;column-gap: 2em;}.graph-report .single-column {}@media only screen and (max-width: 600px) {.graph-report .small-columns {column-count: 1;}.graph-report .medium-columns {column-count: 1;}.graph-report .large-columns {column-count: 1;}}@media only screen and (min-width: 600px) and (max-width: 800px) {.graph-report .small-columns {column-count: 2;}.graph-report .medium-columns {column-count: 1;}.graph-report .large-columns {column-count: 1;}}@media only screen and (min-width: 800px) and (max-width: 1024px) {.graph-report .small-columns {column-count: 3;}.graph-report .medium-columns {column-count: 2;}.graph-report .large-columns {column-count: 1;}}</style><h2>KGCOVID19</h2><p class="paragraph">The undirected multigraph KGCOVID19 has 574.23K heterogeneous nodes and 18.25M heterogeneous edges. The graph contains 30.98K connected components (of which 30.18K are disconnected nodes), with the largest one containing 540.07K nodes and the smallest one containing a single node. The RAM requirements for the nodes and edges data structures are 80.99MB and 73.61MB respectively.</p><h3 style="margin: 1em 0 0 0;">Degree centrality</h3><p class="paragraph">The minimum node degree is 0, the maximum node degree is 122.24K, the mode degree is 1, the mean degree is 63.57 and the node degree median is 4.</p><p class="paragraph">The nodes with the highest degree centrality are <a href='https://ncbi.nlm.nih.gov/mesh/?term=D014780' target='_blank' title='Go to NCBI Mesh to get more informations about MESH:D014780'>MESH:D014780</a> (degree 122.24K and node type <a href='https://biolink.github.io/biolink-model/docs/OntologyClass.html' target='_blank' title='Go to BioLink to get more informations about biolink:OntologyClass'>biolink:OntologyClass</a>), <a href='https://ncbi.nlm.nih.gov/mesh/?term=D006801' target='_blank' title='Go to NCBI Mesh to get more informations about MESH:D006801'>MESH:D006801</a> (degree 104.31K and node type <a href='https://biolink.github.io/biolink-model/docs/OntologyClass.html' target='_blank' title='Go to BioLink to get more informations about biolink:OntologyClass'>biolink:OntologyClass</a>), <a href='https://ncbi.nlm.nih.gov/mesh/?term=D018352' target='_blank' title='Go to NCBI Mesh to get more informations about MESH:D018352'>MESH:D018352</a> (degree 92.93K and node type <a href='https://biolink.github.io/biolink-model/docs/OntologyClass.html' target='_blank' title='Go to BioLink to get more informations about biolink:OntologyClass'>biolink:OntologyClass</a>), <a href='https://www.ncbi.nlm.nih.gov/taxonomy/?term=2697049' target='_blank' title='Go to NCBI Taxonomy to get more informations about NCBITaxon:2697049'>NCBITaxon:2697049</a> (degree 92.31K and node types <a href='https://biolink.github.io/biolink-model/docs/OntologyClass.html' target='_blank' title='Go to BioLink to get more informations about biolink:OntologyClass'>biolink:OntologyClass</a>, <a href='https://biolink.github.io/biolink-model/docs/OrganismTaxon.html' target='_blank' title='Go to BioLink to get more informations about biolink:OrganismTaxon'>biolink:OrganismTaxon</a> and <a href='https://biolink.github.io/biolink-model/docs/OrganismalEntity.html' target='_blank' title='Go to BioLink to get more informations about biolink:OrganismalEntity'>biolink:OrganismalEntity</a>) and <a href='https://www.wikidata.org/wiki/Q30' target='_blank' title='Go to WikiData to get more informations about WD:Q30'>WD:Q30</a> (degree 92.14K and node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>).</p><h3 style="margin: 1em 0 

## Connected holdout
Since we want to visualize an edge prediction task on this graph, we need to create a connected holdout:

In [3]:
%%time
train, test = graph.connected_holdout(train_size=0.7)
train.enable()

CPU times: user 22.6 s, sys: 428 ms, total: 23 s
Wall time: 16.3 s


## Compute the embedding
Next, we compute the embedding using the First-order LINE method. Do note that this implementation is a data-race aware one that uses SGD as optimizer, and nothing fancy like ADAM or NADAM: this means that the memory footprint is only limited to the embedding size.

In [4]:
%%time
from grape.embedders import FirstOrderLINEEnsmallen
embedding = FirstOrderLINEEnsmallen().fit_transform(train)

CPU times: user 55min 34s, sys: 2.31 s, total: 55min 36s
Wall time: 2min 22s


/home/luca/anaconda3/lib/python3.9/site-packages/embiggen/utils/abstract_models/abstract_embedding_model.py:166: UserWarning: Please be advised that the KGCOVID19 train graph contains 30179 disconnected nodes. Consider that node embedding algorithms that only use topological information such as CBOW, GloVe, SPINE and SkipGram are not able to provide meaningful embeddings for these nodes, and their embedding will be generally far away from any other node. It is also possible that all disconnected nodes will receive a relatively similar node embedding. Consider dropping them by using the `graph.remove_disconnected_nodes()` method.
  warnings.warn(


## Visualize the embedding on the test graph
We are at the end, finally visualizing the test graph.

In [9]:
from grape import GraphVisualizer

vis = GraphVisualizer(
    graph=test,
    support=train,
    n_components=4,
    edge_embedding_method="Hadamard",
    rotate=True,
    verbose=True,
    # Automatically, since LINE learns a cosine, the visualization tool
    # would dispatch a Cosine-distance based TSNE. This would use the sklearn
    # implementation, which is terribly slow. Therefore, we force it to use the Euclidean distance
    # and therefore the Multicore TSNE implementation (when available).
    decomposition_kwargs=dict(metric="euclidean")
)

Then we run the TSNE, this may take a while.

In [10]:
%%time
vis.fit_negative_and_positive_edges(embedding)

/home/luca/anaconda3/lib/python3.9/site-packages/embiggen/visualizations/graph_visualizer.py:557: UserWarning: Please do be advised that when using a node embedding method such as Glove, which embeds nodes using a dot product, it is highly suggested to use a `cosine` metric. Using a different metric, such as the one you have provided (euclidean) may lead to worse decompositions using UMAP or t-SNE.
  warnings.warn(
Performing t-SNE using 24 cores.
Using no_dims = 4, perplexity = 30.000000, and theta = 0.500000
Computing input similarities...
Building tree...
 - point 2003 of 20000
 - point 4000 of 20000
 - point 6001 of 20000
 - point 8001 of 20000
 - point 10000 of 20000
 - point 14886 of 20000
 - point 15787 of 20000
 - point 16000 of 20000
 - point 18000 of 20000
 - point 20000 of 20000
Done in 1.00 seconds (sparsity = 0.006050)!
Learning embedding...
Iteration 51: error is 102.628107 (50 iterations in 5.00 seconds)
Iteration 101: error is 88.360756 (50 iterations in 7.00 seconds)
I

CPU times: user 6min 37s, sys: 13min 40s, total: 20min 18s
Wall time: 51.8 s


Fitting performed in 51.00 seconds.


In [13]:
%%time
vis.plot_positive_and_negative_edges()

Rendering frames:   0%|                                                                                       …

OpenCV: FFMPEG: tag 0x30387076/'vp80' is not supported with codec id 139 and format 'webm / WebM'


Merging frames:   0%|                                                                                         …

CPU times: user 9min 48s, sys: 12.2 s, total: 10min 1s
Wall time: 1min 37s
